In [1]:
from fls_functions import fls_fog

In [2]:
some_text = 'Scott? Helen? Okay. Are you there, Scott?'
fls_fog(some_text)

{'fl_sents': 0,
 'nfl_sents': 4,
 'fl_fog': None,
 'fl_fog_sents': None,
 'fl_complex_words': None,
 'fl_fog_words': None,
 'nfl_fog': 0.7000000000000001,
 'nfl_complex_words': 0,
 'nfl_fog_words': 7,
 'nfl_fog_sents': 4,
 'nfl_positive': 0,
 'nfl_negative': 0,
 'nfl_uncertainty': 0,
 'nfl_litigious': 0,
 'nfl_modal_strong': 0,
 'nfl_modal_weak': 0}

In [3]:
some_text = """Welcome to the Manitowoc Co. Inc. second quarter's earnings conference call. 
Today's call is being recorded.
There will be a question and answer session following the presentation.
Instructions for asking a question will be given at that time.
For opening remarks and introductions I would like to turn the call over to Mr. Kyle.
You may begin, Sir.
"""

In [4]:
fls_fog(some_text)

{'fl_sents': 3,
 'nfl_sents': 3,
 'fl_fog': 8.082051282051282,
 'fl_complex_words': 3,
 'fl_fog_words': 26,
 'fl_fog_sents': 3,
 'nfl_fog': 11.5921568627451,
 'nfl_complex_words': 6,
 'nfl_fog_words': 34,
 'nfl_fog_sents': 3,
 'fl_positive': 0,
 'fl_negative': 2,
 'fl_uncertainty': 1,
 'fl_litigious': 0,
 'fl_modal_strong': 2,
 'fl_modal_weak': 1,
 'nfl_positive': 0,
 'nfl_negative': 0,
 'nfl_uncertainty': 0,
 'nfl_litigious': 0,
 'nfl_modal_strong': 0,
 'nfl_modal_weak': 0}

In [5]:
import pandas as pd
from sqlalchemy import create_engine

conn_string = 'postgresql://localhost/crsp'

engine = create_engine(conn_string)

sql =  """
    SELECT file_name, last_update, speaker_number, context, section,
       speaker_text
    FROM streetevents.speaker_data
    WHERE file_name = '1000000_T' 
    ORDER BY section, context, speaker_number
"""

speaker_data = pd.read_sql(sql, engine)

In [6]:
import json

def expand_json(df, col):
    return pd.concat([df.drop([col], axis=1),
                      df[col].map(lambda x: json.loads(json.dumps(x))).apply(pd.Series)], axis=1)

In [7]:
speaker_data['last_update'] = speaker_data['last_update'].map(lambda x: str(x.astimezone('UTC')))
speaker_data['fls_data'] = speaker_data['speaker_text'].apply(fls_fog)
speaker_data = expand_json(speaker_data, 'fls_data')
speaker_data = speaker_data.drop(['speaker_text'], axis=1)

In [8]:
speaker_data

,file_name,last_update,speaker_number,context,section,fl_sents,nfl_sents,fl_fog,fl_complex_words,fl_fog_words,...,fl_uncertainty,fl_litigious,fl_modal_strong,fl_modal_weak,nfl_positive,nfl_negative,nfl_uncertainty,nfl_litigious,nfl_modal_strong,nfl_modal_weak
0,1000000_T,2005-07-29 19:37:34+00:00,1,pres,1,3.0,3.0,8.082051,3.0,26.0,...,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000000_T,2005-07-29 19:37:34+00:00,2,pres,1,9.0,7.0,17.178744,40.0,230.0,...,3.0,1.0,6.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1000000_T,2005-07-29 19:37:34+00:00,3,pres,1,15.0,31.0,14.650090,50.0,296.0,...,7.0,0.0,6.0,1.0,16.0,5.0,2.0,1.0,1.0,0.0
3,1000000_T,2005-07-29 19:37:34+00:00,4,pres,1,11.0,22.0,13.308204,30.0,205.0,...,5.0,0.0,5.0,4.0,9.0,6.0,3.0,2.0,0.0,0.0
4,1000000_T,2005-07-29 19:37:34+00:00,5,pres,1,17.0,23.0,13.961420,51.0,333.0,...,5.0,0.0,7.0,2.0,15.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1000000_T,2005-07-29 19:37:34+00:00,74,qa,1,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
79,1000000_T,2005-07-29 19:37:34+00:00,75,qa,1,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
80,1000000_T,2005-07-29 19:37:34+00:00,76,qa,1,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
81,1000000_T,2005-07-29 19:37:34+00:00,77,qa,1,1.0,0.0,8.133333,1.0,12.0,...,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
